# Proxy

A Proxy is an object that stands in place for another object, typically wrapping it up and forwarding calls to it while performing some additional functionality on top.

In [1]:
class PartProxy:
    """ A proxy class for parts - keeping their running counts and a registry """
    
    # Keep count of total number of parts
    parts_count = 0
    register = {}
    
    # A proxy almost always wraps up the object it proxies to
    def __init__(self, part):
        self.part = part
        self.__class__.parts_count += 1
        count = self.__class__.parts_count
        # Register the part!
        self.__class__.register[count] = part
        print(self.register)
        
    def __str__(self):
        return str(self.part)
    
    @classmethod
    def get_count(cls):
        return cls.parts_count
    
    def destroy(self):
        """ destroy the part """

        # Unregister the part
        for key,item in self.__class__.register.items():
            if item == self.part:
                del self.__class__.register[key]
                break
                
        del self.part
        self.__class__.parts_count -= 1
        
    def __getattr__(self, name):
        """ Redirect attributes to proxied part """
        
        try:
            return self.__dict__[name]
        except KeyError:
            return getattr(self.part, name)

In [2]:

class PartsTypeProxyFactory(type):
    """ A type and proxy factory for mechanical part classes and instances respectively """
    
    def __init__(cls, *args, **kwargs):
        print('__init__:',cls)
        type.__init__(cls, *args)
        
    def my_new(cls,name,bases=(),dct={}):
        print(cls,name)
        instance = object.__new__(cls)
        # instance = object.__new__(cls)
        return instance
       
    @classmethod
    def create(cls, *args, **kwargs):
        print('create:',args)
        return PartProxy(Part(*args, **kwargs))
    
    @classmethod
    def __prepare__(cls, name, bases, **kwargs):
        return {'__new__': cls.my_new}


class Part(metaclass=PartsTypeProxyFactory):
    """ A Mechanical Parts class """

    def __init__(self, name, parent=None):
        print('__init__:',name,parent)
        self.name = name
        self.parent = parent
    
    def join(self, part):
        print('Joining with part', part)
        
    def __str__(self):
        return "{}, parent: {} ".format(self.name, self.parent)


__init__: <class '__main__.Part'>


In [3]:
nut = PartsTypeProxyFactory.create('Nut')
bolt = PartsTypeProxyFactory.create('Bolt', nut)
screw = PartsTypeProxyFactory.create('Screw', bolt)
        

create: ('Nut',)
<class '__main__.Part'> Nut
__init__: Nut None
{1: <__main__.Part object at 0x7fb5744a3f28>}
create: ('Bolt', <__main__.PartProxy object at 0x7fb5744a3fd0>)
<class '__main__.Part'> Bolt
__init__: Bolt Nut, parent: None 
{1: <__main__.Part object at 0x7fb5744a3f28>, 2: <__main__.Part object at 0x7fb5744a3a58>}
create: ('Screw', <__main__.PartProxy object at 0x7fb5744a3978>)
<class '__main__.Part'> Screw
__init__: Screw Bolt, parent: Nut, parent: None  
{1: <__main__.Part object at 0x7fb5744a3f28>, 2: <__main__.Part object at 0x7fb5744a3a58>, 3: <__main__.Part object at 0x7fb5744a3dd8>}


In [4]:
PartProxy.get_count()

3

In [5]:
PartProxy.register

{1: <__main__.Part at 0x7fb5744a3f28>,
 2: <__main__.Part at 0x7fb5744a3a58>,
 3: <__main__.Part at 0x7fb5744a3dd8>}

In [6]:
print(nut)
print(bolt)
print(screw)

Nut, parent: None 
Bolt, parent: Nut, parent: None  
Screw, parent: Bolt, parent: Nut, parent: None   


In [7]:
frame = PartsTypeProxyFactory.create('Frame')
nut.join(frame)

create: ('Frame',)
<class '__main__.Part'> Frame
__init__: Frame None
{1: <__main__.Part object at 0x7fb5744a3f28>, 2: <__main__.Part object at 0x7fb5744a3a58>, 3: <__main__.Part object at 0x7fb5744a3dd8>, 4: <__main__.Part object at 0x7fb574490e48>}
Joining with part Frame, parent: None 


In [8]:
nut.destroy()

In [9]:
PartProxy.get_count()

3

In [10]:
bolt.destroy()

In [11]:
PartProxy.get_count()

2

In [12]:
# Check the parts register
PartProxy.register

{3: <__main__.Part at 0x7fb5744a3dd8>, 4: <__main__.Part at 0x7fb574490e48>}

In [13]:
screw.destroy()

In [14]:
PartProxy.get_count()

1

In [15]:
PartProxy.register

{4: <__main__.Part at 0x7fb574490e48>}

In [16]:
frame.destroy()

In [17]:
PartProxy.get_count()

0

In [18]:
PartProxy.register

{}

# Adapter

An adapter adapts an existing class (implementation) to a new interface (class). It does this by either using inheritance (class adapter) or by using aggregation (object adapter)

In [19]:
from abc import abstractmethod

class InvalidPolygonError(Exception):
    pass

class Polygon:
    """ A generic polygon class """
    
    def __init__(self, *sides):
        """ Initializer - accepts length of sides """
        self.sides = sides
        
    def perimeter(self):
        """ Return perimeter """
        
        return sum(self.sides)
    
    @abstractmethod
    def is_valid(self):
        """ Is this a valid polygon """
        pass
    
    def is_regular(self):
        """ Is a regular polygon ? """
        
        # Yes: if all sides are equal
        side = self.sides[0]
        return all([x==side for x in self.sides[1:]])
    
    @abstractmethod
    def area(self):
        """ Calculate and return area """
        
        pass


In [20]:
class Rectangle(Polygon):
    """ Rectangle class from Polygon using class adapter """
        
    def is_square(self):
        """ Return if I am a square """

        if self.is_valid():
            # Defaults to is_regular
            return self.is_regular()

    def is_valid(self):
        """ Is the rectangle valid """

        # Should have 4 sides
        if len(self.sides) != 4:
            return False

        # Opposite sides should be same
        for a,b in [(0,2),(1,3)]:
            if self.sides[a] != self.sides[b]:
                return False

        return True

    def area(self):
        """ Return area of rectangle """

        # Length x breadth
        if self.is_valid():
            return self.sides[0]*self.sides[1]

In [21]:
rect = Rectangle(10, 4, 10, 4)

print(rect.is_valid())
print(rect.is_square())
print(rect.perimeter())
print(rect.area())

True
False
28
40


In [22]:
def rectangle_valid(self):
    """ Is the rectangle valid """
    
    # Should have 4 sides
    if len(self.sides) != 4:
        return False

    # Opposite sides should be same
    for a,b in [(0,2),(1,3)]:
        if self.sides[a] != self.sides[b]:
            return False

    return True
    
def triangle_valid(self):
    """ Is the triangle valid """
        
    # Sum of 2 sides should be > 3rd side
    perimeter = self.perimeter()
    for side in self.sides:
        sum_two = perimeter - side
        if sum_two <= side:
            return False
        
    return True
                
class PolygonType(type):
    """ A generic polygon type """
    
    def my_init(self, *sides):
        """ Initializer - accepts length of sides """
        self.sides = sides
        
    def my_perimeter(self):
        """ Return perimeter """
        
        return sum(self.sides)
    
    def my_is_regular(self):
        """ Is a regular polygon ? """
        
        # Yes: if all sides are equal
        side = self.sides[0]
        return all([x==side for x in self.sides[1:]])
        
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('Metaclass=>',mcs, name)
            
        # Attach is_valid method using class name
        class_name = name.lower()
        valid_func = eval(class_name + '_valid')
        print('Attaching function',valid_func,'as is_valid method')
        return {'__init__': mcs.my_init, 'perimeter': mcs.my_perimeter, 
               'is_regular': mcs.my_is_regular, 'is_valid': valid_func}
       

In [23]:
import itertools

class Rectangle(metaclass=PolygonType):
    """ Rectangle class using metaclasses """
    
    def area(self):
        """ Return area of rectangle """
        
        print('Calculating area')
        # Length x breadth
        if self.is_valid():
            return self.sides[0]*self.sides[1]
        
class Triangle(metaclass=PolygonType):
    """ Triangle class using metaclasses """
    
    def is_isosceles(self):
        """ Is the triangle isoscles """
        
        if self.is_valid():
            # Check if any 2 sides are equal
            for a,b in itertools.combinations(self.sides, 2):
                if a == b:
                    return True
        return False
    
    def area(self):
        """ Calculate area """
        
        # Using Heron's formula
        p = self.perimeter()/2.0
        total = p
        for side in self.sides:
            total *= abs(p-side)
            
        return pow(total, 0.5)

Metaclass=> <class '__main__.PolygonType'> Rectangle
Attaching function <function rectangle_valid at 0x7fb57443e1e0> as is_valid method
Metaclass=> <class '__main__.PolygonType'> Triangle
Attaching function <function triangle_valid at 0x7fb57443e400> as is_valid method


In [24]:
rect2 = Rectangle(10,4,10,4)
print(rect2.is_valid())
print(rect2.area())
print(rect2.is_regular())

True
Calculating area
40
False


In [25]:
tri = Triangle(10, 10, 20)
print(tri.is_valid())

False


In [26]:
tri = Triangle(12,  12, 6)
print(tri.is_valid())
print(tri.is_isosceles())
print(tri.area())

True
True
34.85685011586675
